In [1]:
import re, jieba, os, random
import math
import numpy as np

class NaiveBayesClassifier():
    """
    读取数据集，按10折交叉验证将数据集随机均分为10份，分别处理好评集与差评集。
    """
    def __init__(self, pos_dir, neg_dir):
        global cn_stopwords
        cn_stopwords = []
        with open('sw_for_comments/cn_stopWords.txt') as f:
            for line in f:
                cn_stopwords.append(line.strip())
        
        print("正在准备训练和测试数据，请稍后...")
        self.pos_split = self.process_file_list(pos_dir)
        self.neg_split = self.process_file_list(neg_dir)
        self.jieba_add()

        
    def jieba_add(self):
        """
        增加分词库的词语搭配
        """
        jieba.add_word('不好看', freq=None, tag=None)
        jieba.add_word('不喜欢', freq=None, tag=None)
        jieba.add_word('不觉得好看', freq=None, tag=None)
        jieba.add_word('哪里好看', freq=None, tag=None)
        

    def get_file_list(self, basedir):
        """
        遍历 basedir 目录下所有 txt 文件，组成一个列表
        """
        file_list = []
        for parent, dirnames, filenames in os.walk(basedir):
            for filename in filenames:
                # 取扩展名
                extension = filename.split('.')[-1]
                if extension == 'txt':
                    file_list.append(os.path.join(parent, filename))
        return file_list


    def process_file_list(self, basedir):
        """
        处理所有 txt 文件
        得到该目录下所包含的评论总数
        将评论集均分为 10 份
        """
        file_list = self.get_file_list(basedir)
        
        all_comments = []
        for file in file_list:
            all_comments += self.get_comment_list(file)
        
        random.shuffle(all_comments) # 将数组中的元素随机排序
        
        comments_split = []
        split_len = len(all_comments) // 10

        print("---正在将数据集随机排序并均等分割为 10 份---")
        for i in range(10):
            start = i * split_len
            end = (i + 1) * split_len
            comments_split.append(all_comments[start:end])

        return comments_split
    
    
    def get_comment_list(self, txt):
        """
        输入：txt格式的评论文件，一行一条评论；
        输出：评论列表，列表的元素个数就是该文件包含的评论数量。
        """
        comment_list = []
        
        with open(txt) as f:
            for line in f:
                if line.strip(): # 检测该行是否为空
                    comment_list.append(line.strip())
        return comment_list

    
    def build_dataset(self, comment_list):
        """
        传入两个参数：评论列表
        提取每条影评的特征词，
        并计算每个词的出现次数，存在 dict 结构中
        """
        feature_words_dict = dict()
        
        for comment in comment_list:
            feature_words = self.find_feature_words(comment)
            for word in feature_words:
                try:
                    feature_words_dict[word] +=1
                except KeyError:
                    """
                    若该词还未出现在词典中，则新建条目，
                    并初始化频数为 1
                    """
                    feature_words_dict[word] = 1
                    
        return feature_words_dict
    
    
    def find_feature_words(self, comment):
        """
        输入：单条评论
        处理：取中文，分词，去停用词
        输出：特征词列表
        """
        pattern = re.compile(u'[\u4e00-\u9fa5]+')
        comment = re.findall(pattern, comment) #匹配中文字符
        comment = ''.join(comment) #将列表连接成一个字符串
        
        temp = jieba.cut(comment, cut_all=False)
        w_list = ' '.join(temp).split()
        feature_list = [w for w in w_list if w not in cn_stopwords]
        return feature_list
    

    def train_validate(self, pos_split, neg_split):
        """
        输入：划分好的数据集（10份）
        处理：按朴素贝叶斯算法进行训练，使用十折交叉验证处理测试数据，计算准确率和召回率
        输出：模型的性能评估
        """
        index = 0
        # 初始化 P，R 数组
        P = [0 for i in range(10)]
        R = [0 for i in range(10)]
        
        while(True):
            print("******正在进行第 %s 次验证******" % str(index + 1))
            """
            ---变量说明---
            test_dataset: 测试集
            pos_split[index]: 测试集中的好评集
            neg_split[index]: 测试集中的差评集
            """
            #test_dataset = pos_split[index] + neg_split[index]
            
            pos_train = []
            neg_train = []
            for i in range(10):
                if i != index:
                    pos_train += pos_split[i]
                    neg_train += neg_split[i]
                    
            print("正在计算词频字典...")
            pos_feature_words_dict = self.build_dataset(pos_train)
            neg_feature_words_dict = self.build_dataset(neg_train)
            print("正在构建词语的好评权重...")
            pos_weights = self.compute_words_weights(pos_feature_words_dict, neg_feature_words_dict)
            
            print("正在对测试集进行验证")
            TP = 0 # 真正例
            FP = 0 # 假正例
            FN = 0 # 假反例
            
            for review in pos_split[index]:
                result = self.classify(review, pos_weights)
                if result == 1:
                    TP += 1
                elif result == 0:
                    FN += 1
                    
            for review in neg_split[index]:
                result = self.classify(review, pos_weights)
                if result == 1:
                    FP += 1
            
            print("正在计算准确率和召回率...")
            P[index] = round((TP / (TP + FP)) * 100, 6)
            R[index] = round((TP / (TP + FN)) * 100, 6)
            print("准确率为：%s" % str(P[index]))
            print("召回率为：%s" % str(R[index]))
            
            index += 1
            if index > 9:
                break
                
        print("10折交叉验证后的准确率均值为：%s" % str(sum(P) / 10))
        print("10折交叉验证后的召回率均值为：%s" % str(sum(R) / 10))
            
    
            
    def classify(self, review, words_weight):
        """
        －－－影评分类器－－－

        先对单条影评进行分词，去除停用词，提取特征词。
        从训练好的权重表中取每个词的好评权重，将权重相加，
        若和大于 0，则属于好评，否则属于差评。

        """
        feature_list = self.find_feature_words(review)

        # 计算该条影评的好评权重值
        pos_weight_sum = 0
        for feature in feature_list:
            """
            在权重表中搜索特征词的权重，若找不到条目，则视为在训练集中未出现过的特征；
            """
            if words_weight.get(feature):
                pos_weight_sum += words_weight.get(feature)
            else:
                pos_weight_sum += words_weight.get('new')

        pos_weight_sum += words_weight.get('pos_cate')

        # 1 代表好评，0 代表差评
        if pos_weight_sum > 0:
            return 1
        else:
            return 0

        
    
    
    def compute_words_weights(self, pos_dict, neg_dict):
        """
        1. 输入：好评特征集、差评特征集；
        2. 处理：先分别对好评集和差评集计算词频，
            再计算每个词语的权重，组成一张权重表，考虑词语不存在的情况；

        3. 默认计算词语属于好评的权重，判断的时候权重和 > 0，则属于好评；
        4. 对重复词语采用多项式模型，并作平滑处理。
        """
#         # 计算特征集中不重复的词语个数，为平滑处理做准备
#         pos_feature_words_sum = len(pos_dict)
#         neg_feature_words_sum = len(neg_dict)
        
        # 计算训练集中的词汇量大小，为平滑处理做准备
        # 首先筛选出只出现在好评集中的词汇，再与差评的词汇表相加，得到训练集的词汇表
        pos_vocabulary = [w for w in pos_dict.keys() if w not in neg_dict.keys()]
        vocab_size = len(pos_vocabulary) + len(neg_dict)

        # 统计全部好评特征词出现次数的总和
        pos_values_all = sum(pos_dict.values())
        # 统计全部差评特征词出现次数的总和
        neg_values_all = sum(neg_dict.values())

        # 处理零概率事件
        not_exist_pos = 1 / (pos_values_all + vocab_size)
        not_exist_neg = 1 / (neg_values_all + vocab_size)

        # 处理好评特征词典，输出词频
        pos_prob_dic = dict()
        for key, value in pos_dict.items():
            prob = (value + 1) / (pos_values_all + vocab_size)
            pos_prob_dic[key] = prob

        # 处理差评特征词典，输出词频
        neg_prob_dic = dict()
        for key, value in neg_dict.items():
            prob = (value + 1) / (neg_values_all + vocab_size)
            neg_prob_dic[key] = prob


        # 构建词语权重表
        weight_table = dict()
        for word, pos_prob in pos_prob_dic.items():
            if neg_prob_dic.get(word):
                neg_prob = neg_prob_dic.get(word)
            else:
                neg_prob = not_exist_neg
            weight_table[word] = math.log(pos_prob / neg_prob)


        for word, neg_prob in neg_prob_dic.items():
            if not weight_table.get(word): # 若权重表中该词语已存在，
                if pos_prob_dic.get(word):
                    pos_prob = pos_prob_dic.get(word)
                else:
                    pos_prob = not_exist_pos
                weight_table[word] = math.log(pos_prob / neg_prob)

        weight_table['new'] = math.log(not_exist_pos / not_exist_neg)

        weight_table['pos_cate'] = math.log(95610 / 75432) # 计算好评集的权重
        
        return weight_table

In [2]:
test = NaiveBayesClassifier('dataset/pos/', 'dataset/neg/')

正在准备训练和测试数据，请稍后...
---正在将数据集随机排序并均等分割为 10 份---


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/ym/q2pw4gf16j33yn01zdhqbhnr0000gn/T/jieba.cache


---正在将数据集随机排序并均等分割为 10 份---


Loading model cost 1.242 seconds.
Prefix dict has been built succesfully.


In [3]:
# 拉普拉斯修正
test.train_validate(test.pos_split, test.neg_split)

******正在进行第 1 次验证******
正在计算词频字典...
正在构建词语的好评权重...
正在对测试集进行验证
正在计算准确率和召回率...
准确率为：82.574003
召回率为：84.09647
******正在进行第 2 次验证******
正在计算词频字典...
正在构建词语的好评权重...
正在对测试集进行验证
正在计算准确率和召回率...
准确率为：82.446809
召回率为：83.974135
******正在进行第 3 次验证******
正在计算词频字典...
正在构建词语的好评权重...
正在对测试集进行验证
正在计算准确率和召回率...
准确率为：82.397782
召回率为：83.117791
******正在进行第 4 次验证******
正在计算词频字典...
正在构建词语的好评权重...
正在对测试集进行验证
正在计算准确率和召回率...
准确率为：82.742521
召回率为：83.624607
******正在进行第 5 次验证******
正在计算词频字典...
正在构建词语的好评权重...
正在对测试集进行验证
正在计算准确率和召回率...
准确率为：83.125864
召回率为：84.026564
******正在进行第 6 次验证******
正在计算词频字典...
正在构建词语的好评权重...
正在对测试集进行验证
正在计算准确率和召回率...
准确率为：82.673438
召回率为：83.930444
******正在进行第 7 次验证******
正在计算词频字典...
正在构建词语的好评权重...
正在对测试集进行验证
正在计算准确率和召回率...
准确率为：82.480687
召回率为：83.965397
******正在进行第 8 次验证******
正在计算词频字典...
正在构建词语的好评权重...
正在对测试集进行验证
正在计算准确率和召回率...
准确率为：83.089445
召回率为：83.8518
******正在进行第 9 次验证******
正在计算词频字典...
正在构建词语的好评权重...
正在对测试集进行验证
正在计算准确率和召回率...
准确率为：82.616534
召回率为：84.09647
******正在进行第 10 次验证******
正在计算词频字典...
正在构建